# Spark Core Demo

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('My First Spark application') \
    .getOrCreate() 

sc = spark.sparkContext

# RDDs (Week 2)

In [2]:
lines = sc.textFile('../non_auto_assignments/data/misc/blurb.txt')

In [3]:
data_lines = lines.filter(lambda x: 'data' in x)

In [4]:
data_lines.take(10)

['The University of Michigan School of Information is pleased to offer a new, fully online, master’s program in applied data science. We teach comprehensive applied data science at the intersection of people and technology. ',
 'We provide critical insight into data collection, computation, and analytics, and help develop hands-on skills using a multidisciplinary approach embedded in information, computer science, and statistics. ',
 'Coursework and projects focus on applying data science to real-world problems. ']

In [5]:
data_lines.first()

'The University of Michigan School of Information is pleased to offer a new, fully online, master’s program in applied data science. We teach comprehensive applied data science at the intersection of people and technology. '

In [6]:
%%timeit
nums = sc.parallelize([1, 2, 3, 4])
squared = nums.map(lambda x: x * x).collect()
for num in squared:
    print('%i ' % (num))


1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 
1 
4 
9 
16 

In [7]:
import multiprocessing
import psutil
print("processor count: " + str(multiprocessing.cpu_count()))
psutil.virtual_memory()

processor count: 32


svmem(total=267535601664, available=250691035136, percent=6.3, used=14644326400, free=15406735360, active=53809348608, inactive=191485493248, buffers=1043898368, cached=236440641536, shared=12288, slab=5554008064)

In [8]:
#Sometimes we want to produce multiple output elements for each input element. The operation to do this is called flatMap(). As with map(), the function we provide to flatMap() is called individually for each element in our input RDD. Instead of returning a single element, we return an iterator with our return values. Rather than producing an RDD of iterators, we get back an RDD that consists of the elements from all of the iterators.

lines = sc.parallelize(["hello world", "hi", "hi world", "hello everyone", "hello"])
words = lines.flatMap(lambda line: line.split(" "))
print(words.collect()) 

word_map = lines.map(lambda line:line.split(' '))
print(word_map.collect()) 

word_counts = words.map(lambda word: (word,1))
print(word_counts.collect())

['hello', 'world', 'hi', 'hi', 'world', 'hello', 'everyone', 'hello']
[['hello', 'world'], ['hi'], ['hi', 'world'], ['hello', 'everyone'], ['hello']]
[('hello', 1), ('world', 1), ('hi', 1), ('hi', 1), ('world', 1), ('hello', 1), ('everyone', 1), ('hello', 1)]


In [9]:
wc = word_counts.reduceByKey(lambda accumulator,value: accumulator + value)

In [10]:
wc.take(10)

[('hello', 3), ('hi', 2), ('world', 2), ('everyone', 1)]

In [11]:
wc.sortByKey(lambda x,y: x+y).collect()

[('everyone', 1), ('hello', 3), ('hi', 2), ('world', 2)]

In [12]:
wc.sortBy(lambda x: x[1], ascending=False).take(10)

[('hello', 3), ('hi', 2), ('world', 2), ('everyone', 1)]

In [13]:
business = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_business.json.gz')
checkin = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_checkin.json.gz')
review = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_review.json.gz')
tip = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_tip.json.gz')
user = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_user.json.gz')

In [14]:
review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [15]:
result = user.filter(user['compliment_cool'] > 5000).collect()

In [16]:
checkin.first()

Row(business_id='--1UhMGODdWsrMastO9DZw', date='2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02')

In [17]:
checkin.dtypes

[('business_id', 'string'), ('date', 'string')]

In [18]:
checkin.groupby(['weekday','hour'])

In [19]:
from pyspark.sql.functions import udf

In [20]:
from pyspark.sql.types import IntegerType
square_udf_int = udf(lambda z: square(z), IntegerType())

In [21]:
from pyspark.sql.types import ArrayType,StringType
datesplit = udf(lambda x: x.split(','),ArrayType(StringType()))

In [22]:
(checkin.select('business_id',datesplit('date').alias('dates'))).first()

Row(business_id='--1UhMGODdWsrMastO9DZw', dates=['2016-04-26 19:49:16', ' 2016-08-30 18:36:57', ' 2016-10-15 02:45:18', ' 2016-11-18 01:54:50', ' 2017-04-20 18:39:06', ' 2017-05-03 17:58:02'])

In [23]:
from pyspark.sql.functions import explode

In [24]:
checkin.select('business_id',datesplit('date').alias('dates')).withColumn('checkin_date',explode('dates')).take(10)

[Row(business_id='--1UhMGODdWsrMastO9DZw', dates=['2016-04-26 19:49:16', ' 2016-08-30 18:36:57', ' 2016-10-15 02:45:18', ' 2016-11-18 01:54:50', ' 2017-04-20 18:39:06', ' 2017-05-03 17:58:02'], checkin_date='2016-04-26 19:49:16'),
 Row(business_id='--1UhMGODdWsrMastO9DZw', dates=['2016-04-26 19:49:16', ' 2016-08-30 18:36:57', ' 2016-10-15 02:45:18', ' 2016-11-18 01:54:50', ' 2017-04-20 18:39:06', ' 2017-05-03 17:58:02'], checkin_date=' 2016-08-30 18:36:57'),
 Row(business_id='--1UhMGODdWsrMastO9DZw', dates=['2016-04-26 19:49:16', ' 2016-08-30 18:36:57', ' 2016-10-15 02:45:18', ' 2016-11-18 01:54:50', ' 2017-04-20 18:39:06', ' 2017-05-03 17:58:02'], checkin_date=' 2016-10-15 02:45:18'),
 Row(business_id='--1UhMGODdWsrMastO9DZw', dates=['2016-04-26 19:49:16', ' 2016-08-30 18:36:57', ' 2016-10-15 02:45:18', ' 2016-11-18 01:54:50', ' 2017-04-20 18:39:06', ' 2017-05-03 17:58:02'], checkin_date=' 2016-11-18 01:54:50'),
 Row(business_id='--1UhMGODdWsrMastO9DZw', dates=['2016-04-26 19:49:16', 

In [25]:
d1.dtypes

NameError: name 'd1' is not defined

In [26]:
result[0].asDict()

{'average_stars': 4.22,
 'compliment_cool': 12706,
 'compliment_cute': 512,
 'compliment_funny': 12706,
 'compliment_hot': 34167,
 'compliment_list': 90,
 'compliment_more': 626,
 'compliment_note': 3247,
 'compliment_photos': 13065,
 'compliment_plain': 6788,
 'compliment_profile': 418,
 'compliment_writer': 1166,
 'cool': 21624,
 'elite': '2011,2012,2013,2014,2015,2016,2017',
 'fans': 1055,
 'friends': 'Rjd9Td7yYOqu0A1e2KDAyA, 4UCqhtKZgLQqp7Ns9VcPdQ, ODSH1it5oAUPu2clIgOvyg, xxneUsUNNYdbr2Ukh_yQlw, BSVThVjjFaF5orzkW7Of7Q, K6v0XrtRVVfkTndN3Dn6UA, EPK4_bIsyVlrlsLH3RClEQ, kRcYmBUumNiDH6iHg3GgOg, 9x3Jo-__QEK_8jwRTZ1zig, vVYZmlco7x4KK_wY-wO1cg, lU9HJlDot9CPsVdnyrANWQ, MiTp_htT7xH7kdAixL_Hzw, Qz3qWko-7VEaWS2tKuc9Aw, MFI4twbiz8UJxU-XHSlH-g, 07JaiREcp2197QiRCR-YHA, Dqb34bnL4dzwZvySx--qqQ, VNRW7STLvplnmo1Akpy2xA, IxKVvGmFIEz77YYL-5_YxA, 0XXIv9a0LWiaCjAkurjMMQ, yi9eAPNgnqBGaRQ_Eyz2Aw, Fw4UjJ6yBeyPB27Y4wwEUQ, C85i4kcR8Ckt-n9zy3H0Tg, BfoIyb0JdWYQPWNKbqWT5w, QzPoBEYZ_nJFwiXOCBcXVw, NZtFNfHJB_hGwbd

In [27]:
review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [28]:
tip.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- compliment_count: long (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)



In [29]:
checkin.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)



In [30]:
business.columns

['address',
 'attributes',
 'business_id',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'name',
 'postal_code',
 'review_count',
 'stars',
 'state']

In [31]:
business.dtypes

[('address', 'string'),
 ('attributes',
  'struct<AcceptsInsurance:string,AgesAllowed:string,Alcohol:string,Ambience:string,BYOB:string,BYOBCorkage:string,BestNights:string,BikeParking:string,BusinessAcceptsBitcoin:string,BusinessAcceptsCreditCards:string,BusinessParking:string,ByAppointmentOnly:string,Caters:string,CoatCheck:string,Corkage:string,DietaryRestrictions:string,DogsAllowed:string,DriveThru:string,GoodForDancing:string,GoodForKids:string,GoodForMeal:string,HairSpecializesIn:string,HappyHour:string,HasTV:string,Music:string,NoiseLevel:string,Open24Hours:string,OutdoorSeating:string,RestaurantsAttire:string,RestaurantsCounterService:string,RestaurantsDelivery:string,RestaurantsGoodForGroups:string,RestaurantsPriceRange2:string,RestaurantsReservations:string,RestaurantsTableService:string,RestaurantsTakeOut:string,Smoking:string,WheelchairAccessible:string,WiFi:string>'),
 ('business_id', 'string'),
 ('categories', 'string'),
 ('city', 'string'),
 ('hours',
  'struct<Friday:st

In [34]:
business.describe().show()

+-------+-------------------+--------------------+--------------------+---------+-------------------+-----------------+------------------+--------------------+------------------+------------------+------------------+------+
|summary|            address|         business_id|          categories|     city|            is_open|         latitude|         longitude|                name|       postal_code|      review_count|             stars| state|
+-------+-------------------+--------------------+--------------------+---------+-------------------+-----------------+------------------+--------------------+------------------+------------------+------------------+------+
|  count|             192609|              192609|              192127|   192609|             192609|           192609|            192609|              192609|            192609|            192609|            192609|192609|
|   mean|               null|                null|                null|     null| 0.8230404602069478|38.

In [35]:
business.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

I'm checking to see if this is the same server...